In [2]:
# Jenny branch
import numpy as np

import tensorflow as tf 

from sklearn import linear_model

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Flatten, BatchNormalization

from sklearn.model_selection import KFold

BORDER = "----------------------------------------------------------------------------"

Using TensorFlow backend.


In [3]:
def load_data(filename, skiprows = 1):
    """
    Function loads data stored in the file filename and returns it as a numpy ndarray.
    
    Inputs:
        filename: given as a string.
        
    Outputs:
        Data contained in the file, returned as a numpy ndarray
    """
    return np.loadtxt(filename, skiprows=skiprows, delimiter=' ')

In [15]:
def ClassifyWithNeuralNetwork(X_train, Y_train, X_test):
    """
    Function takes training and testing data, and fits using our neural network implementation
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
    """
    print('\n{}\nNEURAL NETWORK\n{}\n'.format(BORDER, BORDER))

    # Converting Y values to one hot vector
    num_classes = 2
    Y_train = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    # Input size
    n_train = X_train.shape[0]
    n_words = X_train.shape[1]
    n_test = Y_train.shape[0]
    
    # Layer set up.
    model = Sequential()
    model.add(Dense(200, input_shape=(n_words,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.15))

    model.add(Dense(100, input_shape=(n_words,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.15))

    model.add(Dense(50, input_shape=(n_words,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.15))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    ## Printing a summary of the layers and weights in your model
    model.summary()

    ## In the line below we have specified the loss function as 'mse' (Mean Squared Error) because in the above code we did not one-hot encode the labels.
    ## In your implementation, since you are one-hot encoding the labels, you should use 'categorical_crossentropy' as your loss.
    ## You will likely have the best results with RMS prop or Adam as your optimizer.  In the line below we use Adadelta
    model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])

    fit = model.fit(X_train, Y_train, batch_size=128, epochs=10, verbose=1)
    
    ## Printing the accuracy of our model, according to the loss function specified in model.compile above
    score = model.evaluate(X_train, Y_train, verbose=0)
    print('Training score:', score[0])
    print('Training accuracy:', score[1])

    prediction = model.predict(X_test, verbose=1)

    zeros = prediction[:, 0]
    ones  = prediction[:, 1]

    prediction = (zeros < ones).astype(int)

    return model, prediction, score[1]

In [16]:
def ClassifyWithConvolutionNetwork(X_train, Y_train, X_test):
    """
    Function takes training and testing data, and fits using our neural network implementation
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
    """
    print('\n{}\nNEURAL NETWORK\n{}\n'.format(BORDER, BORDER))

    # Converting Y values to one hot vector
    num_classes = 2
    Y_train = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    # Input size
    n_train = X_train.shape[0]
    n_words = X_train.shape[1]
    n_test = Y_train.shape[0]
    
    
    # For 1D Convolution, change dimension of input 
    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)
    
        
    ## Create your own model here given the constraints in the problem
    model = Sequential()

    model.add(Conv1D(5, 3, padding="same", input_shape=(n_words, 1, )))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))

    model.add(Conv1D(5, 2, padding="same"))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.1))

    model.add(Flatten())
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))

    ## Once you one-hot encode the data labels, the line below should be predicting probabilities of each of the 2 classes
    model.add(Dense(2))
    model.add(Activation('softmax'))

    ## Printing a summary of the layers and weights in your model
    model.summary()

    ## In the line below we have specified the loss function as 'mse' (Mean Squared Error) because in the above code we did not one-hot encode the labels.
    ## In your implementation, since you are one-hot encoding the labels, you should use 'categorical_crossentropy' as your loss.
    ## You will likely have the best results with RMS prop or Adam as your optimizer.  In the line below we use Adadelta
    model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])

    fit = model.fit(X_train, Y_train, batch_size=128, epochs=10, verbose=1)
    
    ## Printing the accuracy of our model, according to the loss function specified in model.compile above
    score = model.evaluate(X_train, Y_train, verbose=0)
    print('Training score:', score[0])
    print('Training accuracy:', score[1])

    prediction = model.predict(X_test, verbose=1)

    zeros = prediction[:, 0]
    ones  = prediction[:, 1]

    prediction = (zeros < ones).astype(int)

    return model, prediction, score[1]

In [17]:
def ClassifyWithLogisticRegression(X_train, Y_train, X_test):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
    """
    
    print('\n{}\nLOGISTIC REGRESSION\n{}\n'.format(BORDER, BORDER))
    
    model = linear_model.LogisticRegression()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    print('Training accuracy: ', accuracy)
    
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [18]:
def CrossValidation(X_train, Y_train, X_test):
    kf = KFold(n_splits = 5)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
     # Converting Y values to one hot vector
    num_classes = 2
    Y = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    
    # For 1D Convolution, change dimension of input 
    X = np.expand_dims(X_train, axis=2)
    
    
    # perform 5-fold validation
    for i in range(0,5):
        traini, vali = inds[i]
        model, prediction, accuracy = ClassifyWithConvolutionNetwork(X_train[traini], Y_train[traini],X_test)
        
                
        # Compute accuracy.
        val_acc = model.evaluate(X[vali], Y[vali], verbose=1)
            
        total_train_acc = np.append(total_train_acc, accuracy)
        total_val_acc = np.append(total_val_acc, val_acc[1])

    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / 5.)
    print("average val accuracy", np.sum(total_val_acc) / 5.)
    
    return 0


In [8]:
# Loads training dataset
training = load_data('data/training_data.txt', 1)
X_train = training[:, 1:]
Y_train = training[:,0]

In [9]:
# Loads testing dataset
# There is no label for testing set 
X_test = load_data('data/test_data.txt', 1)

In [19]:
# Using Validation 
CrossValidation(X_train, Y_train, X_test)


----------------------------------------------------------------------------
NEURAL NETWORK
----------------------------------------------------------------------------

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_15 (Conv1D)           (None, 1000, 5)           20        
_________________________________________________________________
batch_normalization_8 (Batch (None, 1000, 5)           20        
_________________________________________________________________
activation_29 (Activation)   (None, 1000, 5)           0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 1000, 5)           0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 1000, 5)           55        
_________________________________________________________________
activation_30 (Activation)   (None, 1

Epoch 1/10
16000/16000 [==============================] - 12s 749us/step - loss: 0.5592 - acc: 0.7401
Epoch 2/10
16000/16000 [==============================] - 11s 699us/step - loss: 0.3946 - acc: 0.8250
Epoch 3/10
16000/16000 [==============================] - 12s 751us/step - loss: 0.3351 - acc: 0.8549
Epoch 4/10
16000/16000 [==============================] - 11s 701us/step - loss: 0.2900 - acc: 0.8789
Epoch 5/10
16000/16000 [==============================] - 11s 705us/step - loss: 0.2384 - acc: 0.9019
Epoch 6/10
16000/16000 [==============================] - 11s 694us/step - loss: 0.2000 - acc: 0.9213
Epoch 7/10
16000/16000 [==============================] - 11s 696us/step - loss: 0.1615 - acc: 0.9368
Epoch 8/10
16000/16000 [==============================] - 11s 716us/step - loss: 0.1361 - acc: 0.9476
Epoch 9/10
16000/16000 [==============================] - 11s 716us/step - loss: 0.1157 - acc: 0.9579
Epoch 10/10
16000/16000 [==============================] - 11s 703us/step - loss: 

4000/4000 [==============================] - 1s 315us/step
training accuracy [0.9904375 0.996625  0.9958125 0.9899375 0.99025  ]
val accuracy [0.818   0.80325 0.80525 0.817   0.80775]
average training accuracy 0.9926124999999999
average val accuracy 0.8102499999999999


0

In [ ]:
model, prediction, accuracy = ClassifyWithNeuralNetwork(X_train, Y_train, X_test)
model, prediction, accuracy  = ClassifyWithConvolutionNetwork(X_train, Y_train, X_test)
model, prediction, accuracy  = ClassifyWithLogisticRegression(X_train, Y_train, X_test)

In [ ]:
print(prediction)
print("prediction shape is: {}".format(prediction.shape))

output = "Id,Prediction\n"
for i in range(prediction.shape[0]):
    output = output + ("{0},{1}\n".format(i + 1, prediction[i]))

file = open('predictions.csv','w') 
file.write(output)
file.close()